# GOURMET GUIDE

### Foundation Project 1: Food Recommendation Model

#### Group 6

* Charanjeet Singh - 12220064
* Pooja Nilesh Doshi - 12220028
* Snigdha Debashis Bhattacharjee - 12220067
* Vinayak Dave - 12220047

In [1]:
import streamlit as st
import pickle
import pandas as pd
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_excel(r"C:\Users\Green Lantern\OneDrive\Documents\ISB AMPBA Winter 2023\Term 2\Foundational Project 1\FP1_Project 4_Group6_Final\restaurant dataset\zomato2.xlsx")

In [3]:
#Deleting Unnnecessary Columns
train_df = train_df.drop(['Unnamed: 0', 'url'],axis=1)

In [4]:
train_df.set_index('name', inplace=True)

In [5]:
indices = pd.Series(train_df.index)

In [6]:
 cost_clusters = {
    0: [100.0, 250.0],
    1: [300.0, 550.0],
    2: [600.0, 950.0]}
train_df['cost_cluster']=''
train_df.loc[(train_df['cost'] >= 100.0) & (train_df['cost'] <= 250.0), 'cost_cluster'] = 0
train_df.loc[(train_df['cost'] >= 300.0) & (train_df['cost'] <= 550.0), 'cost_cluster'] = 1
train_df.loc[(train_df['cost'] >= 600.0) & (train_df['cost'] <= 950.0), 'cost_cluster'] = 2

In [7]:
def main():
    st.title("GOURMET GUIDE")
    user_recommendation()

In [8]:
def user_recommendation():
    unique_loc = train_df['location'].unique()
    location = st.selectbox("Choose Location:", unique_loc)

    unique_cuisines_df = train_df[train_df['location'] == location]['cuisines']
    unique_cuisines = unique_cuisines_df.unique()
    if unique_cuisines_df.empty:
        st.warning("Choose your Location")
        return

    cuisine = st.selectbox("Choose Cuisine:", unique_cuisines)

    Rest_Name_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine)]
    if Rest_Name_df.empty:
        st.warning("Choose your preferred Cuisine")
        return
    Rest_Name = Rest_Name_df['rest_type'].unique()
    rest_type = st.selectbox("Choose Restaurant Type:", Rest_Name)

    filtered_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine) & (train_df['rest_type'] == rest_type)]
    if filtered_df.empty:
        st.warning("No Restaurants Found")
        return

    cost_cluster = []
    ranges = []
    cost_cluster = filtered_df['cost_cluster'].unique()

    for i in cost_cluster:
        for cluster, cost_range in cost_clusters.items():
            if cluster == i:
                ranges.append(f"{cluster}. Rs.{cost_range[0]} to Rs.{cost_range[1]}")
    
    cost_range = st.selectbox("Cost for Two:", reversed(ranges))

    cost_range = int(cost_range.split(".")[0])
    cost_filter = (filtered_df['cost'] >= cost_clusters[cost_range][0]) & (filtered_df['cost'] <= cost_clusters[cost_range][1])
    filtered_df = filtered_df[cost_filter]

    if filtered_df.empty:
        st.warning("No Restaurants Found")
        return

    #Preprocessing 'reviews_list' column to handle missing values
    filtered_df['reviews_list'].fillna('', inplace=True)
    
    num_rows = filtered_df.shape[0]
    if num_rows == 1:
        name = filtered_df.index.tolist()
        recommendations_df = pd.DataFrame({'Restaurant': name, 'Rating': filtered_df['rate']})
        recommendations_df = recommendations_df.reset_index(drop=True)
    else:
        tfidf_reviews = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
        tfidf_reviews_matrix = tfidf_reviews.fit_transform(filtered_df['reviews_list'])

        cosine_reviews = cosine_similarity(tfidf_reviews_matrix, tfidf_reviews_matrix)

        idx = filtered_df.index[0]
        unique_indexes = filtered_df.index.unique()

        if idx in unique_indexes:
            idx = unique_indexes.get_loc(idx)
            sim_scores_reviews = list(enumerate(cosine_reviews[idx]))
            sim_scores_reviews = sorted(list(sim_scores_reviews), key=lambda x: x[1], reverse=True)
            top5_indexes_reviews = [i[0] for i in sim_scores_reviews[1:6]]
            recommend_restaurants_reviews = list(filtered_df.index[top5_indexes_reviews].unique())
            recommend_ratings_reviews = [np.unique(filtered_df.loc[restaurant]['rate']) for restaurant in recommend_restaurants_reviews]

        if not recommend_restaurants_reviews:
            st.warning("No Restaurants Found")
            return

        recommendations_df = pd.DataFrame({'Restaurant': recommend_restaurants_reviews, 'Rating': recommend_ratings_reviews})

    st.success("Restaurants you may also like:")
    st.table(recommendations_df)

    selected_restaurant = st.selectbox("Select a restaurant from the list above:", recommendations_df['Restaurant'].values)

    if selected_restaurant not in recommendations_df['Restaurant'].values:
        st.warning("Invalid restaurant selection")
        return

    selected_index = recommendations_df[recommendations_df['Restaurant'] == selected_restaurant].index[0]
    selected_recommend_dishes = filtered_df.loc[recommendations_df.loc[selected_index, 'Restaurant'], 'dish_liked']
    
    if isinstance(selected_recommend_dishes, str):
        selected_recommend_dishes = [selected_recommend_dishes]
    else:
        selected_recommend_dishes = selected_recommend_dishes.drop_duplicates().tolist()

    all_dishes = ','.join(selected_recommend_dishes)
    dishes_list = [dish.strip() for dish in all_dishes.split(',')]
    unique_dishes = list(set(dishes_list))
    df = pd.DataFrame({'Dishes': unique_dishes})

    st.success(f"Dishes you will like in {selected_restaurant}:")
    st.write(df['Dishes'])

In [9]:
if __name__ == "__main__":
    main()

2023-06-17 23:39:18.732 
  command:

    streamlit run C:\Users\Green Lantern\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
